In [17]:
from graphpy.graph import Graph

folder_name = "data/"

"""
read the data
"""
# json files transfer to csv files
import jsonlines
num_vertex = 2000


In [33]:
# Create a new graph
import numpy as np
d_gp=Graph(directed=True)
gp=Graph()
random_list = np.random.randint(1, 1000000, size=(num_vertex))
random_list.sort()
print(random_list)
# add a new vertex
jsonName = folder_name + 'yelp_academic_dataset_user.json'
with open(jsonName, 'r', encoding='UTF-8') as f:
    i = 0
    j = 0
    k = 0
    for item in jsonlines.Reader(f):
        if len(gp.get_all_vertex()) > num_vertex:
            break
        k += 1
        if random_list[j] != k:
            continue
        user_id = item["user_id"]
        if gp.get_vertex(user_id) == None:
            gp.add_vertex(user_id, [i])
            d_gp.add_vertex(user_id, [i])
            i = i + 1
        friends = item["friends"].split(", ")
        for friend in friends:
            if gp.get_vertex(friend) == None:
                gp.add_vertex(friend, [i])
                d_gp.add_vertex(friend, [i])
                i = i + 1
            sc = gp[user_id]
            dt = gp[friend]
            sc1 = d_gp[user_id]
            dt1 = d_gp[friend]
            if dt1 not in d_gp.adjacents_vertex(sc1):
                d_gp.add_edge(sc1, dt1)
            if sc1 not in d_gp.adjacents_vertex(dt1):
                d_gp.add_edge(dt1, sc1)
            if (dt not in gp.adjacents_vertex(sc)) and (sc not in gp.adjacents_vertex(dt)):
                gp.add_edge(sc, dt)
        # gp.print_adjacent_list()
        j += 1

# to get the create vertex, you can use the [] operator
# print(gp.get_all_vertex())
# print(gp.get_all_edges())
d_gp.print_adjacent_list()
gp.print_adjacent_list()


[   125    247   1164 ... 998827 998850 999979]
{D3ZMD1KEub2EjPkdpW9EvA: [gPdPwZpSIlE8FCPWn5Wftw, e2PRcG5K_a6TkeBrt9h8VQ, PiIxAF5eA_jloMbcvPiqIQ, LZ-3gZouNIvMmb8ixGJpRw, I2oY25XBtfCK9msTaDqzOw, f9YdhGpRc37RAGcBFp7PUg, fEPuPOl8200jHz-1VMcokQ, rZlH42Q9EpKs6bUYokK1sQ, hBcI21RWvB30D0RATopvMg, meQxIRc4pgbd3UilffcKEw, 43aCQzbggstjUwxSF5Kvtg, 3aGWphbjWP5M7XLCR-QTug, ik8BOIlDYsttwQYduRDSXQ, 0KpnAD5p_IRhsoGBJldYGQ, HEIHrFAwVqvUjh63qu3OBg, 03ti7RuF_l-ucP1BUuZMUA, 1vo3UdvBwoX2PCWZBdQ8lQ, fOa1c_cvKjYPSb0Rr6VaNg, D5XYtCdeP2p1V89JbHEvHQ, 3_Xrok7pV_MNBHhCPy2hgg, -7oi4m1R466pphSKV2xpKQ, 2czNUZ2IW-qx_Cd0RHaWiw, CHNVyccWElehmBNxn5Jc7A, XbNelpCgzeAKbotHAiBmuA, 6RZP5vWFdsm8ydfz8hBhkg, f4Zx46zNFLttQ1BCuWl3Lw, B-eSYeYr9JCOn5mV02gchg, zbL_KoNHr0f-cauX_5sqRw, -NFjA1o5Bl73P2EgayiUDQ, fCCi_jPO07l3JYvb_ohRyw, 4a_YQf5JNkqDJhyNv0RPAQ, -faSCP1wd3JXaZ4FqD7bsA, 7DNCMwpfwwwim34SWV9q1g, Z9sU4IKUZDG7zyQgSws6hA, EqmNxqkg9SDUjvw8y0C3qA, VwNYAKZarbCVsdK_GzncUg, vxXWZ_rsww2jWqX5v-7UFA, dvk6ZzOr-caeHtGFeVS76g, JPRxxzFH6U0T3X

In [34]:
_, _, results = gp.search(ConnectedComponentStrategy())
print(len(gp.get_all_vertex()))
print("If we advertise the new ads, we need advertise {} people.".format(results))

2121
If we advertise the new ads, we need advertise 3 people.


In [9]:
from graphpy.search_strategy import SearchStrategy
class ConnectedComponentStrategy(SearchStrategy):
    def __init__(self):

        self.__labels = {}
        self.__label = 0
        self.__firstSee = {}
        self.__close = {}
        self.__time = 0

    def __dfs_visit(self, vertex):
        self.__time = self.__time + 1
        self.__firstSee[vertex] = self.__time
        vertex.set_color(1)

        for adjacent in self.get_adjacent_list()[vertex]:
            if adjacent.get_color() == 0:
                self.__labels[adjacent] = self.__label
                self.__dfs_visit(adjacent)
        vertex.set_color(2)
        self.__time += 1
        self.__close[vertex] = self.__time

    def search(self):
        # colors:
        #   white: not visited
        #   grey: in the queue
        #   black: nothing more to do
        for key in self.get_adjacent_list():
            # set color for all vertices to white
            key.set_color(0)
            self.__labels[key] = None
        self.__time = 0
        self.__label = 0
        for key in self.get_adjacent_list():
            if key.get_color() == 0 and self.__labels[key] == None:
                self.__labels[key] = self.__label
                self.__dfs_visit(key)
                self.__label += 1

        return self.__firstSee, self.__close, self.__label

In [ ]:
class PRIterator:

    def __init__(self, gp):
        self.damping_factor = 0.85  # alpha
        self.max_iterations = 100  # the max times of the iteration
        self.min_delta = 0.00001  # the flag of the iteration end, delta
        self.graph = gp

    def pageRank(self):
        #  we add all edges from the node whose out degree is 0
        for node in self.graph.get_all_vertex():
            if len(self.graph.adjacents_vertex(node)) == 0:
                for node2 in self.graph.get_all_vertex():
                    self.graph.add_edge(node, node2)

        nodes = self.graph.get_all_vertex()
        graph_size = len(nodes)

        if graph_size == 0:
            return {}
        page_rank = dict.fromkeys(nodes, 1.0 / graph_size)  # initial the value
        damping_value = (1.0 - self.damping_factor) / graph_size  # (1-alpha) / N

        flag = False
        for i in range(self.max_iterations):
            change = 0
            for node in nodes:
                rank = 0
                for incident_page in self.graph.adjacents_vertex(node):  # traverse all incident pages
                    rank += self.damping_factor * (page_rank[incident_page] / len(self.graph.adjacents_vertex(incident_page)))
                rank += damping_value
                change += abs(page_rank[node] - rank)  # absolution value
                page_rank[node] = rank

            if change < self.min_delta:
                flag = True
                break
        if flag:
            print("finished in %s iterations!" % node)
        else:
            print("finished out of 100 iterations!")
        return page_rank

In [ ]:
pr = PRIterator(d_gp)
result = pr.pageRank()
fresult = sorted(result.items(), key = lambda kv:kv[1])
print("The most famous person in the customers is {}.".format(fresult[0][0]))